In [1]:
import pandas as pd
df_grocery = pd.read_csv('Recognized_Shop_Healthy_Stores.csv')
df_grocery.head()

,Store Name,Street Address,Address,Borough,Zip \nCode,Year Awarded,Program \nWave,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,SIRAH 1720 LLC,535 MARCY AVE,NaN,Brooklyn,11206,2020,8,40.696961,-73.949512,3.0,36.0,257.0,3338591.0,3.017390e+09,Bedford
1,FIRST STOP GROCERY INC,643 MARCY AVE,NaN,Brooklyn,11206,2020,8,40.692552,-73.948632,3.0,36.0,261.0,3049316.0,3.017710e+09,Bedford
2,AANDE DIRECT LLC,40 LEWIS AVE,NaN,Brooklyn,11206,2020,8,40.696576,-73.937777,3.0,36.0,283.0,3042883.0,3.015850e+09,Stuyvesant Heights
3,CENTRAL ORGANIC,67 CENTRAL AVE,NaN,Brooklyn,11206,2020,8,40.701096,-73.930445,4.0,34.0,425.0,3072058.0,3.031640e+09,Bushwick South
4,194 THROOP FOOD CORP,777 PARK AVE,NaN,Brooklyn,11206,2020,8,40.698191,-73.944311,3.0,36.0,257.0,3335580.0,3.017320e+09,Bedford


In [2]:
def remove_empty_lat_long_rows(df):
    # Remove rows where either latitude or longitude is empty (NaN)
    cleaned_df = df_grocery.dropna(subset=['Latitude', 'Longitude'])
    return cleaned_df

# Example usage
cleaned_df = remove_empty_lat_long_rows(df_grocery)
print(cleaned_df)

                      Store Name  Street Address         Address   Borough  \
0                 SIRAH 1720 LLC   535 MARCY AVE             NaN  Brooklyn   
1         FIRST STOP GROCERY INC   643 MARCY AVE             NaN  Brooklyn   
2               AANDE DIRECT LLC    40 LEWIS AVE             NaN  Brooklyn   
3                CENTRAL ORGANIC  67 CENTRAL AVE             NaN  Brooklyn   
4           194 THROOP FOOD CORP    777 PARK AVE             NaN  Brooklyn   
..                           ...             ...             ...       ...   
363          El Barrio Superette            1858   Lexington Ave  New York   
364  New King Deli Grocery Corp.              23  East 115th St.  New York   
365   Little Mexico Meat Grocery            2119       Third Ave  New York   
366        Paniagua Deli Grocery             203  East 115th St.  New York   
367               JJ Food Market             153   East 99th St.  New York   

     Zip \nCode  Year Awarded  Program \nWave   Latitude  Longi

In [3]:
import urllib.request
import json
import math

url = 'https://data.cityofnewyork.us/resource/xyye-rtrs.json'

# Fetch the data from the URL
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())

# Use the data as needed
print(data)

[{'the_geom': {'type': 'Point', 'coordinates': [-73.8472005205491, 40.89470517661004]}, 'objectid': '1', 'name': 'Wakefield', 'stacked': '1', 'annoline1': 'Wakefield', 'annoline2': '', 'annoline3': '', 'annoangle': '0.0', 'borough': 'Bronx'}, {'the_geom': {'type': 'Point', 'coordinates': [-73.82993910812405, 40.87429419303015]}, 'objectid': '2', 'name': 'Co-op City', 'stacked': '2', 'annoline1': 'Co-op', 'annoline2': 'City', 'annoline3': '', 'annoangle': '0.0', 'borough': 'Bronx'}, {'the_geom': {'type': 'Point', 'coordinates': [-73.82780644716419, 40.88755567735082]}, 'objectid': '3', 'name': 'Eastchester', 'stacked': '1', 'annoline1': 'Eastchester', 'annoline2': '', 'annoline3': '', 'annoangle': '0.0', 'borough': 'Bronx'}, {'the_geom': {'type': 'Point', 'coordinates': [-73.90564259591689, 40.895437426903875]}, 'objectid': '4', 'name': 'Fieldston', 'stacked': '1', 'annoline1': 'Fieldston', 'annoline2': '', 'annoline3': '', 'annoangle': '0.0', 'borough': 'Bronx'}, {'the_geom': {'type': 

In [4]:
# Build dictionary

coordinates_to_neighborhood = {}
for neighborhood in data:
    if neighborhood['borough'] == "Manhattan":
        lat = neighborhood['the_geom']['coordinates'][1]
        lon = neighborhood['the_geom']['coordinates'][0]
        coordinates_to_neighborhood[tuple([lat, lon])] = neighborhood['name']

print(coordinates_to_neighborhood)

{(40.87655077879968, -73.91065965862988): 'Marble Hill', (40.715618422314364, -73.99427936255985): 'Chinatown', (40.85190252555309, -73.93690027985242): 'Washington Heights', (40.86768396449919, -73.92121042203905): 'Inwood', (40.82360428481197, -73.94968791883373): 'Hamilton Heights', (40.81693442949784, -73.95738539351888): 'Manhattanville', (40.81597606742418, -73.94321112603912): 'Central Harlem', (40.79224946663037, -73.94418223148531): 'East Harlem', (40.77563857330185, -73.96050763135007): 'Upper East Side', (40.77592984988492, -73.94711784471833): 'Yorkville', (40.76811265828737, -73.95885968813766): 'Lenox Hill', (40.762403730702246, -73.9494705736962): 'Roosevelt Island', (40.7876579985349, -73.9770592363061): 'Upper West Side', (40.773528889421705, -73.9853377700127): 'Lincoln Square', (40.758333686906404, -73.99640768221386): 'Clinton', (40.75469110270627, -73.98166882730311): 'Midtown', (40.74830307725222, -73.97833207924134): 'Murray Hill', (40.74403470674802, -74.0031163

In [5]:
# Get most likely neighborhood

def get_neighborhood(lat, lon):
    closest_distance = math.inf
    closest_coordinates = None
    
    for key in coordinates_to_neighborhood:
        distance = abs(abs(lat) - abs(key[0])) + abs(abs(lon) - abs(key[1]))
        
        if distance < closest_distance:
            closest_distance = distance
            closest_coordinates = key
    
    return coordinates_to_neighborhood[closest_coordinates]

get_neighborhood(40.87655077879968, -73.91065965862988)

'Marble Hill'

In [6]:
# Test

for coordinates in coordinates_to_neighborhood:
    if get_neighborhood(coordinates[0], coordinates[1]) != coordinates_to_neighborhood[coordinates]:
        print("FAIL")

In [7]:
cleaned_df['neighborhood'] = cleaned_df.apply(lambda row: get_neighborhood(row['Latitude'], row['Longitude']), axis=1)

cleaned_df.tail()

<ipython-input-7-f7bddd5dedf3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['neighborhood'] = cleaned_df.apply(lambda row: get_neighborhood(row['Latitude'], row['Longitude']), axis=1)


,Store Name,Street Address,Address,Borough,Zip \nCode,Year Awarded,Program \nWave,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,neighborhood
363,El Barrio Superette,1858,Lexington Ave,New York,10029,2017,5,40.798118,-73.942006,11.0,8.0,182.0,1052235.0,1.016430e+09,East Harlem North,East Harlem
364,New King Deli Grocery Corp.,23,East 115th St.,New York,10029,2017,5,40.799415,-73.945645,11.0,9.0,184.0,1074184.0,1.016210e+09,East Harlem North,East Harlem
365,Little Mexico Meat Grocery,2119,Third Ave,New York,10029,2017,5,40.797752,-73.940150,11.0,8.0,188.0,1052552.0,1.016650e+09,East Harlem North,East Harlem
366,Paniagua Deli Grocery,203,East 115th St.,New York,10029,2017,5,40.797233,-73.940432,11.0,8.0,188.0,1052555.0,1.016650e+09,East Harlem North,East Harlem
367,JJ Food Market,153,East 99th St.,New York,10029,2017,5,40.787686,-73.949230,11.0,5.0,166.0,1051758.0,1.016270e+09,East Harlem South,Carnegie Hill


In [8]:
def print_unique_neighborhoods(cleaned_df):
    unique_neighborhoods = cleaned_df['neighborhood'].unique()
    for neighborhood in unique_neighborhoods:
        print(neighborhood)

# Example usage
print_unique_neighborhoods(cleaned_df)

Lower East Side
Central Harlem
Hamilton Heights
Marble Hill
Washington Heights
Inwood
East Harlem
Carnegie Hill


In [11]:
def count_and_normalize_grocery_stores_by_neighborhood(cleaned_df):
    # Group by neighborhood and count the number of grocery stores in each neighborhood
    grocery_store_count = cleaned_df.groupby('neighborhood')['Store Name'].count().reset_index(name='total_grocery_stores')
    
    # Normalize the 'total_grocery_stores' column:
    grocery_store_count['normalized_grocery_stores'] = (grocery_store_count['total_grocery_stores'] - grocery_store_count['total_grocery_stores'].min()) / (grocery_store_count['total_grocery_stores'].max() - grocery_store_count['total_grocery_stores'].min())
    
    return grocery_store_count

# Example usage
normalized_grocery_store_count = count_and_normalize_grocery_stores_by_neighborhood(cleaned_df)

# Save the DataFrame to a CSV file
normalized_grocery_store_count.to_csv('normalized_grocery_store_count.csv', index=False)

# Set the caption for the DataFrame
styled_table = normalized_grocery_store_count.style.set_caption("Recognized Grocery Stores")
styled_table


,neighborhood,total_grocery_stores,normalized_grocery_stores
0,Carnegie Hill,2,0.000000
1,Central Harlem,39,0.327434
2,East Harlem,64,0.548673
3,Hamilton Heights,9,0.061947
4,Inwood,4,0.017699
5,Lower East Side,110,0.955752
6,Marble Hill,115,1.000000
7,Washington Heights,10,0.070796


In [12]:
from google.colab import files

# Save the DataFrame to a CSV file
normalized_grocery_store_count.to_csv('normalized_grocery_store_count.csv', index=False)

# Download the file to your local system
files.download('normalized_grocery_store_count.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>